In [1]:
import pandas as pd

In [2]:
ava_df = pd.read_csv('AVA.txt', sep=' ', names=['Image ID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Semantic ID 1', 'Semantic ID 2', 'Challenge ID'])
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
1,953619,0,1,5,17,38,36,15,6,5,1,1,22,1396
2,953958,10,7,15,26,26,21,10,8,1,2,1,21,1396
3,954184,0,0,4,8,41,56,10,3,4,0,0,0,1396
4,954113,0,1,4,6,48,37,23,5,2,2,15,21,1396
5,953980,0,3,6,15,57,39,6,1,1,1,22,38,1396


In [3]:
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
import numpy as np
rows_to_delete = []
for i, row in tqdm(ava_df.iterrows()):
    try:
        _ = Image.open('images/{}.jpg'.format(row['Image ID'])).convert('RGB')
    except:
        rows_to_delete.append(i)   

12627it [03:23, 66.47it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20054016 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
26363it [07:20, 73.92it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 9. 
  warnings.warn(str(msg))
34422it [09:35, 42.58it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 842019120 byte

99388it [29:00, 39.22it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
112149it [32:31, 53.86it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 21652 bytes but only got 848. Skipping tag 700
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4898 bytes but only got 848. Skipping tag 34377
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 940 bytes but only got 848. Skipping tag 34675
  " Skipping tag %s" % (size, len(data), tag))
115570it [33:34, 50.48it/s]/usr/local/lib/python3.7/site-package

In [ ]:
proc_ava_df = ava_df.drop(rows_to_delete)
print('Removed {} rows'.format(len(rows_to_delete)))

In [ ]:
proc_ava_df.to_csv('proc_AVA.csv', index=False)

In [ ]:
ava_df = pd.read_csv('proc_AVA.csv')

In [ ]:
ava_df.head()

In [ ]:
import numpy as np

def compute_mean_score(row):
    prob_distr = row[[1,2,3,4,5,6,7,8,9,10]].values
    prob_distr = prob_distr / prob_distr.sum()
    weights = np.arange(10) + 1
    mean_score = (prob_distr * weights).sum()
    return mean_score

In [ ]:
%%time
ava_df['Mean Score'] = ava_df.apply(compute_mean_score, axis=1)

In [ ]:
ava_df.head()

In [ ]:
ava_df.to_csv('proc_AVA.csv', index=False)

In [ ]:
len(ava_df)